In [1]:
import pickle
import random
from utils import emotion_scores
import numpy as np

C:\Users\Dell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
C:\Users\Dell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\pipelines\text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [2]:
beta_values = pickle.load(open('Checkpoints/beta_values.pkl', 'rb'))
bigram_probs = pickle.load(open('Checkpoints/bigram_probs.pkl', 'rb'))
bigram_probs_laplace = pickle.load(open('Checkpoints/bigram_probs_laplace.pkl', 'rb'))
bigram_probs_kneser = pickle.load(open('Checkpoints/bigram_probs_kneser.pkl', 'rb'))
vocab = pickle.load(open('Checkpoints/vocab.pkl', 'rb'))
beta_values_sentence = pickle.load(open('Checkpoints/beta_values_sentence.pkl', 'rb'))

In [3]:
emotion_dict = {'sadness': 0, 'joy': 1, 'love': 2, 'anger': 3, 'fear': 4, 'surprise': 5}

In [18]:
def generate_sentence(emotion, coeff, max_length=20):
    sentence = []
    current_word = '<SOS>'

    for i in range(max_length):
        current_word = generate_next_word(current_word, emotion, coeff)
        if current_word == "" or current_word == '<EOS>':
            break
        sentence.append(current_word)
    
    return " ".join(sentence)

def generate_next_word(current_word, emotion, coeff):
    if current_word not in vocab:
        raise ValueError(f"{current_word} not found in the vocabulary.")

    word_index = vocab.index(current_word)
    if current_word == '<SOS>':
        next_word_probs = bigram_probs_kneser[word_index]
    else:
        next_word_probs = bigram_probs_kneser[word_index] + (coeff * beta_values[word_index, :, emotion_dict[emotion]])
        next_word_probs -= np.min(next_word_probs)
        next_word_probs /= np.max(next_word_probs)


    # next_word_index = list(next_word_probs).index(max(next_word_probs))
    try:
        next_word_index = random.choices(range(len(next_word_probs)), weights=next_word_probs)[0]
    except:
        return ""

    next_word = list(vocab)[next_word_index]

    return next_word

In [17]:
generate_sentence('joy', 0.5)

0.9999999999999999


'i am thankful that honestly feel generally more excitment than inspired because im feelin horny'

#### sample level beta values

In [8]:
for emotion in ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']:
    with open('Test Samples/sample_level/gen_' + emotion + '.txt', 'w') as f:
        for i in range(50):
            while True:
                sentence = generate_sentence(emotion, 1)
                if len(sentence.split()) > 6:
                    f.write(sentence + '\n')
                    break

In [9]:
count = 0
with open('Test Samples/sample_level/gen_sadness.txt', 'r') as f:
    for line in f.readlines():
        score = emotion_scores(line)[emotion_dict['sadness']]['score']
        
        if(score > 0.5):
            count += 1

print('sadness: ' + str(count))

count = 0
with open('Test Samples/sample_level/gen_joy.txt', 'r') as f:
    for line in f.readlines():
        score = emotion_scores(line)[emotion_dict['joy']]['score']
        
        if(score > 0.5):
            count += 1

print('joy: ' + str(count))

count = 0
with open('Test Samples/sample_level/gen_love.txt', 'r') as f:
    for line in f.readlines():
        score = emotion_scores(line)[emotion_dict['love']]['score']
        
        if(score > 0.5):
            count += 1

print('love: ' + str(count))

count = 0
with open('Test Samples/sample_level/gen_anger.txt', 'r') as f:
    for line in f.readlines():
        score = emotion_scores(line)[emotion_dict['anger']]['score']
        
        if(score > 0.5):
            count += 1

print('anger: ' + str(count))

count = 0
with open('Test Samples/sample_level/gen_fear.txt', 'r') as f:
    for line in f.readlines():
        score = emotion_scores(line)[emotion_dict['fear']]['score']
        
        if(score > 0.5):
            count += 1

print('fear: ' + str(count))

count = 0
with open('Test Samples/sample_level/gen_surprise.txt', 'r') as f:
    for line in f.readlines():
        score = emotion_scores(line)[emotion_dict['surprise']]['score']
        
        if(score > 0.5):
            count += 1

print('surprise: ' + str(count))


        


sadness: 4
joy: 12
love: 1
anger: 20
fear: 6
surprise: 1


#### Coeff = 0, smoothing = None

In [11]:
for emotion in ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']:
    with open('Test Samples/coeff_0_no/gen_' + emotion + '.txt', 'w') as f:
        for i in range(50):
            while True:
                sentence = generate_sentence(emotion, 0)
                if len(sentence.split()) > 6:
                    f.write(sentence + '\n')
                    break

In [12]:
count = 0
with open('Test Samples/coeff_0_no/gen_sadness.txt', 'r') as f:
    for line in f.readlines():
        score = emotion_scores(line)[emotion_dict['sadness']]['score']
        
        if(score > 0.5):
            count += 1

print('sadness: ' + str(count))

count = 0
with open('Test Samples/coeff_0_no/gen_joy.txt', 'r') as f:
    for line in f.readlines():
        score = emotion_scores(line)[emotion_dict['joy']]['score']
        
        if(score > 0.5):
            count += 1

print('joy: ' + str(count))

count = 0
with open('Test Samples/coeff_0_no/gen_love.txt', 'r') as f:
    for line in f.readlines():
        score = emotion_scores(line)[emotion_dict['love']]['score']
        
        if(score > 0.5):
            count += 1

print('love: ' + str(count))

count = 0
with open('Test Samples/coeff_0_no/gen_anger.txt', 'r') as f:
    for line in f.readlines():
        score = emotion_scores(line)[emotion_dict['anger']]['score']
        
        if(score > 0.5):
            count += 1

print('anger: ' + str(count))

count = 0
with open('Test Samples/coeff_0_no/gen_fear.txt', 'r') as f:
    for line in f.readlines():
        score = emotion_scores(line)[emotion_dict['fear']]['score']
        
        if(score > 0.5):
            count += 1

print('fear: ' + str(count))

count = 0
with open('Test Samples/coeff_0_no/gen_surprise.txt', 'r') as f:
    for line in f.readlines():
        score = emotion_scores(line)[emotion_dict['surprise']]['score']
        
        if(score > 0.5):
            count += 1

print('surprise: ' + str(count))


        


sadness: 8
joy: 18
love: 9
anger: 9
fear: 4
surprise: 6


#### Coeff = 1, smoothing = None

In [13]:
for emotion in ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']:
    with open('Test Samples/coeff_1_no/gen_' + emotion + '.txt', 'w') as f:
        for i in range(50):
            while True:
                sentence = generate_sentence(emotion, 1)
                if len(sentence.split()) > 6:
                    f.write(sentence + '\n')
                    break

In [14]:
count = 0
with open('Test Samples/coeff_1_no/gen_sadness.txt', 'r') as f:
    for line in f.readlines():
        score = emotion_scores(line)[emotion_dict['sadness']]['score']
        
        if(score > 0.5):
            count += 1

print('sadness: ' + str(count))

count = 0
with open('Test Samples/coeff_1_no/gen_joy.txt', 'r') as f:
    for line in f.readlines():
        score = emotion_scores(line)[emotion_dict['joy']]['score']
        
        if(score > 0.5):
            count += 1

print('joy: ' + str(count))

count = 0
with open('Test Samples/coeff_1_no/gen_love.txt', 'r') as f:
    for line in f.readlines():
        score = emotion_scores(line)[emotion_dict['love']]['score']
        
        if(score > 0.5):
            count += 1

print('love: ' + str(count))

count = 0
with open('Test Samples/coeff_1_no/gen_anger.txt', 'r') as f:
    for line in f.readlines():
        score = emotion_scores(line)[emotion_dict['anger']]['score']
        
        if(score > 0.5):
            count += 1

print('anger: ' + str(count))

count = 0
with open('Test Samples/coeff_1_no/gen_fear.txt', 'r') as f:
    for line in f.readlines():
        score = emotion_scores(line)[emotion_dict['fear']]['score']
        
        if(score > 0.5):
            count += 1

print('fear: ' + str(count))

count = 0
with open('Test Samples/coeff_1_no/gen_surprise.txt', 'r') as f:
    for line in f.readlines():
        score = emotion_scores(line)[emotion_dict['surprise']]['score']
        
        if(score > 0.5):
            count += 1

print('surprise: ' + str(count))


        


sadness: 41
joy: 41
love: 47
anger: 33
fear: 31
surprise: 41


#### Coeff = 1, smoothing = laplace

In [16]:
for emotion in ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']:
    with open('Test Samples/coeff_1_laplace/gen_' + emotion + '.txt', 'w') as f:
        for i in range(50):
            while True:
                sentence = generate_sentence(emotion, 1)
                if len(sentence.split()) > 6:
                    f.write(sentence + '\n')
                    break

In [17]:
count = 0
with open('Test Samples/coeff_1_laplace/gen_sadness.txt', 'r') as f:
    for line in f.readlines():
        score = emotion_scores(line)[emotion_dict['sadness']]['score']
        
        if(score > 0.5):
            count += 1

print('sadness: ' + str(count))

count = 0
with open('Test Samples/coeff_1_laplace/gen_joy.txt', 'r') as f:
    for line in f.readlines():
        score = emotion_scores(line)[emotion_dict['joy']]['score']
        
        if(score > 0.5):
            count += 1

print('joy: ' + str(count))

count = 0
with open('Test Samples/coeff_1_laplace/gen_love.txt', 'r') as f:
    for line in f.readlines():
        score = emotion_scores(line)[emotion_dict['love']]['score']
        
        if(score > 0.5):
            count += 1

print('love: ' + str(count))

count = 0
with open('Test Samples/coeff_1_laplace/gen_anger.txt', 'r') as f:
    for line in f.readlines():
        score = emotion_scores(line)[emotion_dict['anger']]['score']
        
        if(score > 0.5):
            count += 1

print('anger: ' + str(count))

count = 0
with open('Test Samples/coeff_1_laplace/gen_fear.txt', 'r') as f:
    for line in f.readlines():
        score = emotion_scores(line)[emotion_dict['fear']]['score']
        
        if(score > 0.5):
            count += 1

print('fear: ' + str(count))

count = 0
with open('Test Samples/coeff_1_laplace/gen_surprise.txt', 'r') as f:
    for line in f.readlines():
        score = emotion_scores(line)[emotion_dict['surprise']]['score']
        
        if(score > 0.5):
            count += 1

print('surprise: ' + str(count))


        


sadness: 48
joy: 41
love: 46
anger: 38
fear: 36
surprise: 44


#### Coeff = 1, smoothing = kneser ney

In [20]:
for emotion in ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']:
    with open('Test Samples/coeff_1_kneser/gen_' + emotion + '.txt', 'w') as f:
        for i in range(50):
            while True:
                sentence = generate_sentence(emotion, 1)
                if len(sentence.split()) > 6:
                    f.write(sentence + '\n')
                    break

In [21]:
count = 0
with open('Test Samples/coeff_1_kneser/gen_sadness.txt', 'r') as f:
    for line in f.readlines():
        score = emotion_scores(line)[emotion_dict['sadness']]['score']
        
        if(score > 0.5):
            count += 1

print('sadness: ' + str(count))

count = 0
with open('Test Samples/coeff_1_kneser/gen_joy.txt', 'r') as f:
    for line in f.readlines():
        score = emotion_scores(line)[emotion_dict['joy']]['score']
        
        if(score > 0.5):
            count += 1

print('joy: ' + str(count))

count = 0
with open('Test Samples/coeff_1_kneser/gen_love.txt', 'r') as f:
    for line in f.readlines():
        score = emotion_scores(line)[emotion_dict['love']]['score']
        
        if(score > 0.5):
            count += 1

print('love: ' + str(count))

count = 0
with open('Test Samples/coeff_1_kneser/gen_anger.txt', 'r') as f:
    for line in f.readlines():
        score = emotion_scores(line)[emotion_dict['anger']]['score']
        
        if(score > 0.5):
            count += 1

print('anger: ' + str(count))

count = 0
with open('Test Samples/coeff_1_kneser/gen_fear.txt', 'r') as f:
    for line in f.readlines():
        score = emotion_scores(line)[emotion_dict['fear']]['score']
        
        if(score > 0.5):
            count += 1

print('fear: ' + str(count))

count = 0
with open('Test Samples/coeff_1_kneser/gen_surprise.txt', 'r') as f:
    for line in f.readlines():
        score = emotion_scores(line)[emotion_dict['surprise']]['score']
        
        if(score > 0.5):
            count += 1

print('surprise: ' + str(count))


        


sadness: 19
joy: 32
love: 17
anger: 33
fear: 22
surprise: 30
